Sample Deep Convolutional GAN for Image Generation. 
Code Inspired by https://www.tensorflow.org/tutorials/generative/dcgan, https://machinelearningmastery.com/how-to-develop-a-conditional-generative-adversarial-network-from-scratch/, https://arxiv.org/abs/1511.06434v2

In [ ]:
# To generate GIFs
!pip install imageio
!pip install git+https://github.com/tensorflow/docs

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display
import numpy as np
from scipy.spatial.distance import cdist
import tensorflow as tf 

##Import Data
Get the image data, and pre-process it 

In [ ]:
(train_images, train_labels), (_, _) = tf.keras.datasets.fashion_mnist.load_data()

train_images = train_images[train_labels == 7]
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]
print(train_images.shape)

print("exaple item from dataset")
plt.imshow(train_images[0].reshape((28,28)), cmap="gray")
plt.show()

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

##Define Generator
Create the generator network

In [ ]:
def define_generator():
	model = tf.keras.Sequential()
	# start at 7x7 image
	n_nodes = 128 * 7 * 7
	model.add(layers.Dense(n_nodes, input_shape=(100,)))
	model.add(layers.LeakyReLU(alpha=0.2))
	model.add(layers.Reshape((7, 7, 128)))
	# upsample to 14x14
	model.add(layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(layers.LeakyReLU(alpha=0.2))
	# upsample to 28x28
	model.add(layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(layers.LeakyReLU(alpha=0.2))
	# generate
	model.add(layers.Conv2D(1, (7,7), activation='tanh', padding='same'))
	return model
 

In [ ]:
generator = define_generator()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

## Define Discriminator

The discriminator is a CNN-based image classifier.

In [ ]:
def define_discriminator(in_shape=(28,28,1)):
	model = tf.keras.Sequential()
	# downsample
	model.add(layers.Conv2D(128, (3,3), strides=(2,2), padding='same', input_shape=in_shape))
	model.add(layers.LeakyReLU(alpha=0.2))
	# downsample
	model.add(layers.Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(layers.LeakyReLU(alpha=0.2))
	# single output classifier
	model.add(layers.Flatten())
	model.add(layers.Dropout(0.4))
	model.add(layers.Dense(1, activation='sigmoid'))
	return model

Use the discriminator to classify the generated images as real or fake. 

In [ ]:
discriminator = define_discriminator()
print(generated_image.shape)
decision = discriminator(generated_image)
print (decision)

## Define the loss and optimizers


In [ ]:
# General binary cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy()

### Discriminator loss


In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

### Generator loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

The discriminator and the generator optimizers are different since we will train two networks separately.

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002, beta_1=0.5)

## Define the training loop


In [ ]:
EPOCHS = 100
noise_dim = 100
num_examples_to_generate = 16

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

The training loop begins with generator receiving a random seed as input. That seed is used to produce an image. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.

In [ ]:

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

**Generate and save images**


In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  #save image to create gif later
  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

def sample_images_from_dataset(dataset):
  
  for images in dataset.take(1):

    fig = plt.figure(figsize=(4,4))

    for i in range(16):
        plt.subplot(4, 4, i+1)
        plt.imshow(images[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    #save image to create gif later
    plt.show()



In [ ]:
train(train_dataset, EPOCHS)

Restore the latest checkpoint.

## Create a GIF


In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(EPOCHS)
print("The gan generated Images")

In [ ]:
sample_images_from_dataset(train_dataset)
print("The original Images")

Use `imageio` to create an animated gif using the images saved during training.

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:

import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)